<a href="https://colab.research.google.com/github/hyeong777/Python-Quest/blob/main/hair_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 로드
!pip install efficientnet_pytorch
import time
import datetime
import os
import copy
import cv2
import random
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from PIL import Image
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
random_seed = 100
random.seed(random_seed)
torch.manual_seed(random_seed)

In [ ]:
## 학습 컨트롤 부분
# 모델1~6 학습 시 바꿔야 하는 것 (이거 하나만 바꾸면 됨)
train_name = 'model1' # check point //  model 1, 2, 3, 4, 5, 6
# 사용할 모델
model_name = 'efficientnet-b0'  # check point // EfficientNet B0 to B7
# 파라미터
hyper_param_batch = 4  # check point // 배치 사이즈 # 기본 4 이상( > 아웃풋4)
num_epochs = 1 # check point // 에폭 설정

In [ ]:
# 데이터경로
data_train_path ='/content/drive/MyDrive/sdproject/train_data/'+train_name+'/train'
data_validation_path = '/content/drive/MyDrive/sdproject/train_data/'+train_name+'/validation'
# 여기에 모델.pt가 save
PATH = '/content/drive/MyDrive/sdproject/scalp_weights/'

In [ ]:
# 모델
num_classes = 4  # 고정값 아웃풋 0 1 2 3 // 4가지
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes) /
model = model.to(device)

In [ ]:
## 전처리
# 전처리규칙선언
def func(x):
    return x.rotate(90)
transforms_train = transforms.Compose([
    transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BOX),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Lambda(func),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transforms_val = transforms.Compose([
    transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BOX),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# data_train_path 경로의 이미지를 transforms.Compose 로 정규화한 데이터 기준으로 트랜스폼
train_data_set = datasets.ImageFolder(data_train_path, transform=transforms_train)
val_data_set = datasets.ImageFolder(data_validation_path, transform=transforms_val)

In [ ]:
## 로더 : 전처리한 이미지를 라벨링된 정답값과 튜플로 묶는다
dataloaders, batch_num = {}, {}
dataloaders['train'] = DataLoader(train_data_set,
                                  batch_size=hyper_param_batch,
                                  shuffle=True,
                                  num_workers=4)  #  check point  // aihub코드  num_workers = 4
dataloaders['val'] = DataLoader(val_data_set,
                                batch_size=hyper_param_batch,
                                shuffle=False,
                                num_workers=4)  # check point  // aihub코드  num_workers = 4
batch_num['train'], batch_num['val'] = len(train_data_set), len(val_data_set)
print('batch_size : %d,  train/val : %d / %d' % (hyper_param_batch, batch_num['train'], batch_num['val']))

In [ ]:
## 학습 펑션
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    if __name__ == '__main__':
        start_time = time.time()
        since = time.time()
        best_acc = 0.0
        best_model_wts = copy.deepcopy(model.state_dict())
        train_loss, train_acc, val_loss, val_acc = [], [], [], []
        for epoch in tqdm(range(num_epochs)):
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)
            epoch_start = time.time()
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # model.train()  ≫ 모델을 학습 모드로 변환
                else:
                    model.eval()  # model.eval()  ≫ 모델을 평가 모드로 변환
                running_loss = 0.0
                running_corrects = 0
                num_cnt = 0
                for inputs, labels in tqdm(dataloaders[phase]):
                    inputs = inputs.to(device) # 전처리한 이미지데이터
                    labels = labels.to(device) # 이미지의 라벨링한 정답값
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)  # 모델에 인풋(전처리한 이미지데이터)을 넣어서 아웃풋 생성
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)  # 로스 계산
                        if phase == 'train':
                            loss.backward()  # backpropagation
                            optimizer.step()  # weight update
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    num_cnt += len(labels)
                if phase == 'train':
                    scheduler.step()
                epoch_loss = float(running_loss / num_cnt)
                epoch_acc = float((running_corrects.double() / num_cnt).cpu() * 100)
                if phase == 'train':
                    train_loss.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    val_loss.append(epoch_loss)
                    val_acc.append(epoch_acc)
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                if phase == 'val' and epoch_acc > best_acc:
                    best_idx = epoch
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    print('==> best model saved - %d / %.1f' % (best_idx, best_acc))
                epoch_end = time.time() - epoch_start
                print('Training epochs {} in {:.0f}m {:.0f}s'.format(epoch, epoch_end // 60,epoch_end % 60))
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: %d - %.1f' % (best_idx, best_acc))
    model.load_state_dict(best_model_wts)
    torch.save(model, PATH + 'aram_' + train_name + '.pt')  # 모델을 PATH경로에 aram_트레인네임(model1).pt 라는 이름으로 저장한다
    #torch.save(model.state_dict(), PATH + 'president_aram_' + train_name + '.pt')  # 모델의 매개변수 저장
    print('model saved')
    end_sec = time.time() - start_time
    end_times = str(datetime.timedelta(seconds=end_sec)).split('.')
    end_time = end_times[0]
    print("end time :", end_time)
    return model, best_idx, best_acc, train_loss, train_acc, val_loss, val_acc
# def실행할 train_model 파라미터 선언
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
# def 실행
train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=num_epochs)